In [73]:
import pandas as pd
import os
import re
import jieba.posseg as pseg
import jieba
import jieba.analyse
from collections import Counter  #导入collections模块中的counter动作以进行词频统计。
import numpy as np
import csv
from scipy.sparse import coo_matrix
import json

In [74]:
path_dir = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\data'
excel_file = 'liziqi_total_comments_461260.xlsx'
path_file = os.path.join(path_dir,excel_file)
new_words = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\new_words.txt'    #自定义词典。
stop_words = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\stop_words.txt'    #停用词词典。
re_expression = '\\n|\\[.{1,8}\\]|[^\\u4e00-\\u9fa5]'

In [75]:
df_corpus_comments = pd.read_excel(path_file,index_col=0,sheet_name=0)
current_df_comments = pd.DataFrame(df_corpus_comments,columns=['text','user_name','create_time'])  #在原始语料表中选择指定列形成新表。

In [76]:
def clean_text(text):
    pattern = re.compile(f'{re_expression}')
    text = re.sub(pattern,'',str(text))  #这里要把text转换为字符串，否则会报错TypeError: expected string or bytes-like object
    return text

current_df_comments['clean_text'] = current_df_comments['text'].apply(clean_text)

In [77]:
df_comments = current_df_comments.drop(current_df_comments[current_df_comments['clean_text'].map(len) < 2].index )  #删除df_comments表'clean_text'列中字符串长度小于2的列，.index表示此值所在的行索引值。
df_comments.index = df_comments.reset_index(drop=True).index + 1  #从1开始重新设置索引。

In [78]:
jieba.load_userdict(new_words)    #加载自定义词典。
stopwords = [line.strip() for line in open(stop_words, encoding='UTF-8').readlines()]
def cut_word(text,stopwords):
    cut_word = pseg.lcut(text)
    cut_word_pair_list = []  #既包含词又包含词性的列表。
    for word_pair in cut_word:
        if word_pair.word not in stopwords:  #去除停用词，tagging_word.word为取tagging_word这个pair元组里word的值。
            cut_word_pair_list.append(word_pair)
    cut_word_word = ''  #只包含词，不包含词性。
    for word in cut_word_pair_list:
        cut_word_word = cut_word_word + word.word
        cut_word_word = cut_word_word + ' '
    return cut_word_word,cut_word_pair_list

df_comments[['cut_word']],df_comments[['pos_word']] = zip(*df_comments['clean_text'].apply(cut_word,args=(stopwords,)))  #使用zip打包返回结果来处理。返回两个值，同时赋给dataframe中两列。
#df_comments.to_excel(path_dir+'\\'+f'all_comments_%d）.xlsx' % len(df_comments["text"]))  #存入excel文档。
    #2021.5.15 19:34保存

'''  #这里是取pos_word中的值。
s = new_dataframe['pos_word']
for i in s:
    for m in i:
        print(m)  #此时m是个pair。
'''

e:\pycharmprojects\demo0410\venv\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


"  #这里是取pos_word中的值。\ns = new_dataframe['pos_word']\nfor i in s:\n    for m in i:\n        print(m)  #此时m是个pair。\n"

In [79]:
'''
def is_one_word(text): #此函数为寻找单字符。
    one_word_str = ''
    for w in str(text).split():
        if len(w) < 2:
           one_word_str = one_word_str + w
        one_word_str = one_word_str + ' '
    return one_word_str
line  = df_comments['cut_word'].apply(is_one_word)
line_text = (' '.join(line)).split()
line_text = list(set(line_text))
with open('单字.txt','w',encoding='utf-8') as f:
    f.write(' '.join(line_text))
'''

"\ndef is_one_word(text): #此函数为寻找单字符。\n    one_word_str = ''\n    for w in str(text).split():\n        if len(w) < 2:\n           one_word_str = one_word_str + w\n        one_word_str = one_word_str + ' '\n    return one_word_str\nline  = df_comments['cut_word'].apply(is_one_word)\nline_text = (' '.join(line)).split()\nline_text = list(set(line_text))\nwith open('单字.txt','w',encoding='utf-8') as f:\n    f.write(' '.join(line_text))\n"

In [80]:

pos_word_list = []  #将所有词性对添加为一个列表。
for line in df_comments['pos_word']:
    for word in line:
        pos_word_list.append(word)
pos_word_list = list(set(pos_word_list)) #先列表去重，转换为set集合，再重新转换为列表。
pos_word_dict = {}  #将词性list pair转变为字典，但要注意，列表里一个词可能有多个词性（主要是标注的时候模糊了），因此字典的长度要小于列表的长度。字典不能有重复的键，相当于自动去重。
for pair in pos_word_list:
    pair = list(pair)
    pos_word_dict[pair[0]] = pair[1]
#with open(path_dir+'\\'+'all_comments_pos_word_pair_%d.json' % len(pos_word_dict),'w') as file_obj: #将词性字典存入json，以复用或在其它文件中调用。
#     json.dump(pos_word_dict,file_obj)


In [81]:
all_text = ''.join(df_comments['cut_word'])
all_text_list = all_text.split(' ')
word_frq = dict(Counter(all_text_list))
word_frq_sorted = sorted(word_frq.items(), key=lambda x: x[1],reverse=True)

In [82]:
pos = ['ns','nr', 'n', 'nt','nw','nz','vn', 'vd','v','a','ad','an','d','t'] #指定词性种类。
word_frq_list = []
for word in word_frq_sorted:
    if len(word[0]) > 1:  #如果字符长度大于1（非单字）。
        if pos_word_dict[word[0]] in pos:  #如果词性在指定的范围内。
            word_frq_list.append(word)
word_frq_df = pd.DataFrame(word_frq_list,columns=['word','frequency'])
word_frq_df.index = word_frq_df.index + 1
#word_frq_df.to_excel(path_dir+'\\'+'all_comments_word_frq_%d.xlsx' % len(word_frq_df['word']))

In [83]:
jieba.analyse.set_idf_path('idf.txt.big')
keywords = jieba.analyse.extract_tags(all_text,topK=1000,allowPOS=('ns','nr', 'n', 'nt','nw','nz','vn', 'vd','v','a','ad','an','d','t'),withWeight=True)
keywords_df = pd.DataFrame(keywords,columns=['keywords','weight'])
keywords_df.index = keywords_df.index + 1
#keywords_df.to_excel(path_dir+'\\'+'all_comments_keywords_%d.xlsx' % len(keywords_df['keywords']))

In [84]:
filtered_df = df_comments.loc[df_comments['text'].str.contains('生活', na=False)]   #在包括全部数据的表格中进行筛选，形成新表。
filtered_df.index = filtered_df.reset_index(drop=True).index + 1  #重新设置索引。
#filtered_df.to_excel(path_dir+'\\'+'shenghuo_comments_%d.xlsx' % len(filtered_df['text'])) #将筛选后的数据保存到excel文档。

In [85]:
shenghuo_text = ''.join(filtered_df['cut_word'])
shenghuo_text_list = shenghuo_text.split(' ')
shenghuo_frq = dict(Counter(shenghuo_text_list))
shenghuo_frq_sorted = sorted(shenghuo_frq.items(), key=lambda x: x[1],reverse=True)
shenghuo_frq_list = []
for word in shenghuo_frq_sorted:
    if len(word[0]) > 1:  #如果字符长度大于1（非单字）。
        if pos_word_dict[word[0]] in pos:  #如果词性在指定的范围内。\n",
            shenghuo_frq_list.append(word)
shenghuo_frq_df = pd.DataFrame(shenghuo_frq_list,columns=['word','frequency'])
shenghuo_frq_df.index = shenghuo_frq_df.index + 1
#shenghuo_frq_df.to_excel(path_dir+'\\'+'shenghuo_word_frq_%d.xlsx' % len(shenghuo_frq_df['word']))

In [86]:
jieba.analyse.set_idf_path('idf.txt.big')
shenghuo_keywords = jieba.analyse.extract_tags(shenghuo_text,topK=500,allowPOS=('ns','nr', 'n', 'nt','nw','nz','vn', 'vd','v','a','ad','an','d','t'),withWeight=True)
shenghuo_keywords_df = pd.DataFrame(shenghuo_keywords,columns=['keywords','weight'])
shenghuo_keywords_df.index = shenghuo_keywords_df.index + 1
#shenghuo_keywords_df.to_excel(path_dir+'\\'+'shenghuo_keywords_%d.xlsx' % len(shenghuo_keywords_df['keywords']))

